<a href="https://colab.research.google.com/github/sYanXO/NSE-meanReversion-strategy/blob/main/Mean_Reversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:

!pip install ta
!pip install yfinance

import yfinance as yf
import pandas as pd
import ta
import matplotlib.pyplot as plt

tickers = [
    "RELIANCE.NS", "ADANIENT.NS", "HDFCBANK.NS", "ICICIBANK.NS", "INFY.NS", "TCS.NS",
    "LT.NS", "AXISBANK.NS", "BHARTIARTL.NS", "HINDUNILVR.NS", "KOTAKBANK.NS",
    "ITC.NS", "SBIN.NS", "MARUTI.NS", "BAJFINANCE.NS", "ASIANPAINT.NS",
    "TITAN.NS", "WIPRO.NS", "NESTLEIND.NS", "ULTRACEMCO.NS", "NTPC.NS",
    "POWERGRID.NS", "TATAMOTORS.NS", "TECHM.NS", "BRITANNIA.NS", "ONGC.NS",
    "JSWSTEEL.NS", "HCLTECH.NS", "GRASIM.NS", "EICHERMOT.NS", "HEROMOTOCO.NS",
    "ADANIPORTS.NS", "COALINDIA.NS", "APOLLOHOSP.NS", "CIPLA.NS", "DRREDDY.NS",
    "INDUSINDBK.NS", "M&M.NS", "SBILIFE.NS", "TATASTEEL.NS", "TATACONSUM.NS",
    "HDFCLIFE.NS", "BPCL.NS", "BAJAJ-AUTO.NS", "DIVISLAB.NS", "IOC.NS",
    "HDFCBANK.NS", "ADANIENSOL.NS", "ADANIGREEN.NS", "ASIANPAINT.NS",
    "COFORGE.NS", "GODREJCP.NS", "HINDALCO.NS", "INFY.NS", "LTIM.NS",
    "PIDILITIND.NS", "PIIND.NS", "RECLTD.NS", "SBICARD.NS", "SIEMENS.NS",
    "SRF.NS", "SYNGENE.NS", "TATACOMM.NS", "TVSMOTOR.NS", "UNIONBANK.NS",
    "VOLTAS.NS"
]


start_date = "2020-01-01"
end_date = "2024-07-31"

# The optimal parameters we found through our iterative process
window = 20
long_term_window = 100
atr_multiple = 3.0
trend_filter_threshold = 0.04

# A list to store the returns from each individual trade across all stocks
all_portfolio_returns = []

# --- Step 2 & 3: Main Backtesting Loop ---

for i, ticker in enumerate(tickers):
    print(f"\n--- Backtesting {ticker} ({i+1}/{len(tickers)}) ---")

    # 2.1 Data Acquisition for the current ticker
    try:
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if df.empty:
            print(f"No data downloaded for {ticker}. Skipping.")
            continue
        df.columns = df.columns.droplevel(1)
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}. Skipping.")
        continue

    # 2.2 Define the "Mean", Deviation, and Indicators
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['StdDev'] = df['Close'].rolling(window=window).std()
    df['Upper_Band'] = df['SMA'] + (df['StdDev'] * 2)
    df['Lower_Band'] = df['SMA'] - (df['StdDev'] * 2)
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
    df['Long_SMA'] = df['Close'].rolling(window=long_term_window).mean()
    df['ATR'] = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=14).average_true_range()
    df.dropna(inplace=True)

    # 3.1 Implement the Trading Logic
    in_position = False
    buy_price = 0
    trade_log = []

    for j in range(len(df)):
        current_date = df.index[j].date()
        close_price = df['Close'][j]
        lower_band = df['Lower_Band'][j]
        sma = df['SMA'][j]
        long_sma = df['Long_SMA'][j]
        rsi = df['RSI'][j]
        atr = df['ATR'][j]

        is_trending = abs(close_price - long_sma) / long_sma > trend_filter_threshold

        if in_position and (close_price <= buy_price - (atr * atr_multiple)):
            sell_price = close_price
            profit = (sell_price - buy_price) / buy_price
            all_portfolio_returns.append(profit)
            in_position = False
            trade_log.append({'date': current_date, 'action': 'SELL (Dynamic Stop Loss)', 'price': sell_price, 'profit': profit})

        elif not in_position and not is_trending and close_price < lower_band and rsi < 30:
            in_position = True
            buy_price = close_price
            trade_log.append({'date': current_date, 'action': 'BUY', 'price': buy_price})

        elif in_position and close_price >= sma:
            sell_price = close_price
            profit = (sell_price - buy_price) / buy_price
            all_portfolio_returns.append(profit)
            in_position = False
            trade_log.append({'date': current_date, 'action': 'SELL (Reversion to Mean)', 'price': sell_price, 'profit': profit})

    if in_position:
        sell_price = df['Close'].iloc[-1]
        profit = (sell_price - buy_price) / buy_price
        all_portfolio_returns.append(profit)
        trade_log.append({'date': df.index[-1].date(), 'action': 'SELL', 'price': sell_price, 'profit': profit})

    # print(f"  Trades for {ticker}: {len(trade_log)}")

# --- Step 4: Evaluate Portfolio Returns ---

print("\n\n--- Portfolio Backtest Complete ---")
if all_portfolio_returns:
    total_trades = len(all_portfolio_returns)
    cumulative_return = (pd.Series(all_portfolio_returns) + 1).prod() - 1
    win_rate = sum(r > 0 for r in all_portfolio_returns) / total_trades

    print(f"Total trades across all stocks: {total_trades}")
    print(f"Portfolio Cumulative Return: {cumulative_return:.2%}")
    print(f"Portfolio Win Rate: {win_rate:.2%}")
else:
    print("No trades were executed across the entire portfolio.")


--- Backtesting RELIANCE.NS (1/66) ---

--- Backtesting ADANIENT.NS (2/66) ---


/tmp/ipython-input-3549947786.py:50: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date, progress=False)
/tmp/ipython-input-3549947786.py:76: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close_price = df['Close'][j]
/tmp/ipython-input-3549947786.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lower_band = df['Lower_Band'][j]
/tmp/ipython-input-3549947786.py:78: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame


--- Backtesting HDFCBANK.NS (3/66) ---

--- Backtesting ICICIBANK.NS (4/66) ---


/tmp/ipython-input-3549947786.py:50: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date, progress=False)
/tmp/ipython-input-3549947786.py:76: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close_price = df['Close'][j]
/tmp/ipython-input-3549947786.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lower_band = df['Lower_Band'][j]
/tmp/ipython-input-3549947786.py:78: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame


--- Backtesting INFY.NS (5/66) ---

--- Backtesting TCS.NS (6/66) ---


...

/tmp/ipython-input-3549947786.py:76: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close_price = df['Close'][j]
/tmp/ipython-input-3549947786.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lower_band = df['Lower_Band'][j]
/tmp/ipython-input-3549947786.py:78: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sma = df['SMA'][j]
/tmp/ipython-input-3549947786.py:79: FutureWarning: Series.__getitem__ treating keys as positions is depre


--- Backtesting UNIONBANK.NS (65/66) ---


/tmp/ipython-input-3549947786.py:76: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close_price = df['Close'][j]
/tmp/ipython-input-3549947786.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lower_band = df['Lower_Band'][j]
/tmp/ipython-input-3549947786.py:78: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sma = df['SMA'][j]
/tmp/ipython-input-3549947786.py:79: FutureWarning: Series.__getitem__ treating keys as positions is depre


--- Backtesting VOLTAS.NS (66/66) ---


--- Portfolio Backtest Complete ---
Total trades across all stocks: 42
Portfolio Cumulative Return: 143.12%
Portfolio Win Rate: 76.19%


/tmp/ipython-input-3549947786.py:76: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close_price = df['Close'][j]
/tmp/ipython-input-3549947786.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lower_band = df['Lower_Band'][j]
/tmp/ipython-input-3549947786.py:78: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sma = df['SMA'][j]
/tmp/ipython-input-3549947786.py:79: FutureWarning: Series.__getitem__ treating keys as positions is depre